## ESNC Risk and Enforcement Status Analysis

We want to identify 

1. high-risk facilities already under enforcement actions
2. high-risk facilities not under enforcement actions
3. low-risk facilities 

and see if there are any disparate impacts.

In [ ]:
import os

import pandas as pd
import numpy as np 
from datetime import datetime as dt
import matplotlib.pyplot as plt
import sqlalchemy as sa

# set up connection with the database
DB_URI = os.getenv('EPA_DWH')
ENGINE = sa.create_engine(DB_URI)

pd.set_option("display.max_columns", None)

### Read  Data

In [ ]:
# risk scores from 2019 

query = """
SELECT
  *
FROM
  model_outputs.effluent_snc_status_nicole
WHERE
  model_id = 'esnc_selection_bias_2021-09-15_125956_382422'
  AND calendar_quarter_start_date >= timestamp '2018-10-01'
  AND calendar_quarter_start_date <= timestamp '2019-07-01'
"""

with ENGINE.begin() as conn:
    scores = pd.read_sql(query, conn)

scores.head()

In [ ]:
# enforcement data from icis

data_dir = '~/sherlock_oak/EPA/Data/manual'
# icis enforcement records with npdes closed dates
enf = pd.read_csv(os.path.join(data_dir, 'icis_enf_conclusion_2021-09-27.csv'), encoding='latin1')
# icis enforcement and facility cross-walk 
enf_xref = pd.read_csv(os.path.join(data_dir, 'xref_enf_conclusion_facility_2021-09-28.csv'), encoding='latin1')
enf_xref.ICIS_FACILITY_INTEREST_ID = [str(i) for i in enf_xref.ICIS_FACILITY_INTEREST_ID]

In [ ]:
# icis facilities data from icis

query = """
SELECT 
    DISTINCT npdes_permit_id,
    icis_facility_interest_id
FROM 
    icis.facilities
"""

with ENGINE.begin() as conn:
    facilities = pd.read_sql(query, conn)

facilities.head()

### Merge Data

In [ ]:
enf.SETTLEMENT_ENTERED_DATE = pd.to_datetime(enf.SETTLEMENT_ENTERED_DATE)
enf.NPDES_CLOSED_DATE = pd.to_datetime(enf.NPDES_CLOSED_DATE)

In [ ]:
enf_merged = pd.merge(enf, enf_xref[['ICIS_FACILITY_INTEREST_ID', 'ENF_CONCLUSION_ID']], how = 'left')
enf_merged = enf_merged.rename(columns = {'ICIS_FACILITY_INTEREST_ID': 'icis_facility_interest_id'})
enf_merged = pd.merge(enf_merged, facilities, how = 'left')

In [ ]:
print(f"{sum(enf_merged.npdes_permit_id.isna()/len(enf_merged))*100}% enforcement records have no matched npdes permit ids.")
enf_trim = enf_merged[~enf_merged.npdes_permit_id.isna()]

In [ ]:
enf_trim

In [ ]:
# generate an under_enf_flag for each permit-quarter
quarters = scores.calendar_quarter_start_date.unique()
dfs = []
for q in quarters: 
    df = enf_trim[['npdes_permit_id', 'SETTLEMENT_ENTERED_DATE', 'NPDES_CLOSED_DATE']]
    under_enf = df[(df.SETTLEMENT_ENTERED_DATE <= q) & ((df.NPDES_CLOSED_DATE.isna()) | (df.NPDES_CLOSED_DATE >= q))]
    under_enf['calendar_quarter_start_date'] = q
    under_enf['under_enf_flag'] = True
    dfs.append(under_enf)

enf_status = pd.concat(dfs)

In [ ]:
merged = pd.merge(scores[['npdes_permit_id', 'calendar_quarter_start_date', 'score']], enf_status[['npdes_permit_id', 'calendar_quarter_start_date', 'under_enf_flag']], how = 'left')
merged.under_enf_flag = merged.under_enf_flag.fillna(False)
merged

In [ ]:
for i, q in enumerate(quarters):
    df = merged[merged.calendar_quarter_start_date == q]
    plt.title(q)
    plt.hist(df[df.under_enf_flag].score, bins = 30, alpha = 0.5)
    plt.hist(df[~df.under_enf_flag].score, bins = 30, alpha = 0.5)
    plt.legend(['under enf', 'not under enf'])
    plt.xlabel('score')
    plt.ylabel('permit count')
    plt.show()

In [ ]:
merged['predicted_esnc_flag'] = merged.score > 0.39
fac_count = merged.groupby(['predicted_esnc_flag'])['under_enf_flag'].count().to_frame().reset_index()
enf_count = merged.groupby(['predicted_esnc_flag'])['under_enf_flag'].sum().to_frame().reset_index()
fac_count.columns = ['predicted_esnc_flag', 'fac_count']
enf_count.columns = ['predicted_esnc_flag', 'enf_count']
counts = pd.merge(fac_count, enf_count)
counts['under_enf_rate'] = counts.enf_count/counts.fac_count
counts

In [ ]:
# overall
thresholds = np.linspace(0.01, 0.99, 100)
rates = []
for t in thresholds: 
    df = merged.copy()
    df['predicted_esnc_flag'] = df.score > t
    fac_count = df.groupby(['predicted_esnc_flag'])['under_enf_flag'].count().to_frame().reset_index()
    enf_count = df.groupby(['predicted_esnc_flag'])['under_enf_flag'].sum().to_frame().reset_index()
    fac_count.columns = ['predicted_esnc_flag', 'fac_count']
    enf_count.columns = ['predicted_esnc_flag', 'enf_count']
    counts = pd.merge(fac_count, enf_count)
    counts['under_enf_rate'] = counts.enf_count/counts.fac_count
    pred_under_enf_rate = counts[counts.predicted_esnc_flag]['under_enf_rate'][1]
    rates.append(pred_under_enf_rate)

In [ ]:
plt.plot(thresholds, rates)
plt.xlabel('classification thresholds')
plt.ylabel('proportion of predicted ESNC facilities \nunder enforcement actions')
plt.show()

In [ ]:
s = 'CA'
state_df = merged[merged.permit_state == s]
thresholds = np.linspace(0.001, 0.999, 100)
rates = []
for t in thresholds: 
    df = state_df.copy()
    df['predicted_esnc_flag'] = df.score > t
    fac_count = df.groupby(['predicted_esnc_flag'])['under_enf_flag'].count().to_frame().reset_index()
    enf_count = df.groupby(['predicted_esnc_flag'])['under_enf_flag'].sum().to_frame().reset_index()
    fac_count.columns = ['predicted_esnc_flag', 'fac_count']
    enf_count.columns = ['predicted_esnc_flag', 'enf_count']
    counts = pd.merge(fac_count, enf_count)
    counts['under_enf_rate'] = counts.enf_count/counts.fac_count
counts

In [ ]:
# by states
merged['permit_state'] = [i[:2] for i in merged.npdes_permit_id]
states = merged.permit_state.unique()
for s in states: 
    state_df = merged[merged.permit_state == s]
    thresholds = np.linspace(0.001, 0.999, 100)
    rates = []
    for t in thresholds: 
        df = state_df.copy()
        df['predicted_esnc_flag'] = df.score > t
        fac_count = df.groupby(['predicted_esnc_flag'])['under_enf_flag'].count().to_frame().reset_index()
        enf_count = df.groupby(['predicted_esnc_flag'])['under_enf_flag'].sum().to_frame().reset_index()
        fac_count.columns = ['predicted_esnc_flag', 'fac_count']
        enf_count.columns = ['predicted_esnc_flag', 'enf_count']
        counts = pd.merge(fac_count, enf_count)
        counts['under_enf_rate'] = counts.enf_count/counts.fac_count
        try:
            pred_under_enf_rate = counts[counts.predicted_esnc_flag]['under_enf_rate'][1]
        except: 
            pred_under_enf_rate = 0
        rates.append(pred_under_enf_rate)
    plt.title(s)
    plt.plot(thresholds, rates)
    plt.xlabel('classification thresholds')
    plt.ylabel('proportion of predicted ESNC facilities \nunder enforcement actions')
    plt.show()

In [ ]:
data_dir = '~/sherlock_oak/EPA/Data/processed'
merged.to_csv(os.path.join(data_dir, 'esnc_risk_enforcement.csv'))